In [1]:
import geopandas as gpd
from glob import glob
import matplotlib.pyplot as plt
import os
import rioxarray as rio
import xarray as xr
from dem_utils import ArcticDEM
from itertools import product
from dask.distributed import LocalCluster, Client
import numpy as np
import pandas as pd
import utils

In [64]:
dems = glob('../data/id1_*/padded*.tiff')
masks = glob('../data/id1_*/mask*.tiff')
coregd = glob('../data/id1_*/coregd*')
zarrfile = glob('../data/id1_*/*.zarr')
# line = glob('../data/id2_*/*.geojson')
# line = gpd.read_file(line[0])

In [67]:
cluster = LocalCluster()
client = cluster.get_client()

with xr.open_dataset(zarrfile[0],
                     engine='zarr', 
                     ) as ds:
    
    ds = ds['z'].chunk(chunks={'y': len(ds.y) // 4,
                                'x': len(ds.x) // 4})
    
    downsampled = ArcticDEM.downsample(ds, factor=10)
    
    downsampled = downsampled.chunk(
        {'time': -1,
            'y': 'auto',
            'x': 'auto'}
        )

    trend = utils.Trends.make_robust_trend(downsampled,
                                            inp_core_dim='time')

    trend = trend.rename('sec')
    
    trend.attrs = {'description': '''
        theilslope estimates of surface elevation change (sec)
        high_slope and low_slope are the 0.95 confidence interval
        '''}
    
    trend = trend.rio.write_crs(downsampled.rio.crs,
                                        'spatial_ref')

client.shutdown()
cluster.close()

/home/s1759665/micromamba/envs/paper2/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37429 instead
  warnings.warn(


In [87]:
trends = xr.open_dataset('../data/id1_6685x_-3188046y/sec.zarr/', engine='zarr')


In [93]:
trends.rio.set_crs(trends[trends.rio.grid_mapping])

CRSError: Invalid projection: : (Internal Proj Error: proj_create: unrecognized format / unknown name)

In [ ]:
for i in range(len(downsampled.time)):
    fig, ax = plt.subplots()
    downsampled[i,:,:].plot(ax=ax)
    ax.set_title(f'{downsampled.time[i].dt.year.item()}/{downsampled.time[i].dt.month.item()}/{downsampled.time[i].dt.day.item()}')

In [77]:
trends.rio.write_crs?

Signature:
trends.rio.write_crs(
    input_crs: Optional[Any] = None,
    grid_mapping_name: Optional[str] = None,
    inplace: bool = False,
) -> Union[xarray.core.dataset.Dataset, xarray.core.dataarray.DataArray]
Docstring:
Write the CRS to the dataset in a CF compliant manner.

.. warning:: The grid_mapping attribute is written to the encoding.

Parameters
----------
input_crs: Any
    Anything accepted by `rasterio.crs.CRS.from_user_input`.
grid_mapping_name: str, optional
    Name of the grid_mapping coordinate to store the CRS information in.
    Default is the grid_mapping name of the dataset.
inplace: bool, optional
    If True, it will write to the existing dataset. Default is False.

Returns
-------
:obj:`xarray.Dataset` | :obj:`xarray.DataArray`:
    Modified dataset with CF compliant CRS information.

Examples
--------
Write the CRS of the current `xarray` object:

>>> raster.rio.write_crs("epsg:4326", inplace=True)

Write the CRS on a copy:

>>> raster = raster.rio.write_c

In [76]:
trends.rio.set_crs?

Signature: trends.rio.set_crs(input_crs: Any, inplace: bool = True) -> Union[xarray.core.dataset.Dataset, xarray.core.dataarray.DataArray]
Docstring:
Set the CRS value for the Dataset/DataArray without modifying
the dataset/data array.

Parameters
----------
input_crs: object
    Anything accepted by `rasterio.crs.CRS.from_user_input`.
inplace: bool, optional
    If True, it will write to the existing dataset. Default is False.

Returns
-------
:obj:`xarray.Dataset` | :obj:`xarray.DataArray`:
    Dataset with crs attribute.
File:      ~/micromamba/envs/paper2/lib/python3.12/site-packages/rioxarray/rioxarray.py
Type:      method

In [ ]:
dems = []
attrs = []
for f in coregd:
    with rio.open_rasterio(f, chunks='auto') as ds:
        
        if 'to_reg_acqdate1' in ds.attrs.keys():
            _acqdate1 = pd.to_datetime(ds.attrs['to_reg_acqdate1'])
        else:
            _acqdate1 = pd.to_datetime(ds.attrs['ref_acqdate1'])
            
        time_index = xr.DataArray(
            data=_acqdate1, # [pd.to_datetime(ds.attrs['to_register_date'], format="%Y-%m-%d")],
            dims=['band'],
            coords={'band':[1]})
        ds['time'] = time_index
        ds = (ds.swap_dims({'band': 'time'})
              .drop_vars('band')
              .rename('z')
              )
        
        if '_FillValue' in ds.attrs.keys():
            fill_value = ds.attrs['_FillValue']
            ds = xr.where(ds!=fill_value, ds, np.nan, keep_attrs=True)
            ds.attrs['_FillValue'] = np.nan
            dems.append(ds)
        else:
            ds.attrs['_FillValue'] = np.nan
            dems.append(ds)
        
        attrs.append(ds.attrs)

In [ ]:
meta_df = pd.DataFrame(attrs)

# drop some unwanted columns
meta_df.drop(columns=['AREA_OR_POINT', 'scale_factor', 'add_offset',
                      '_FillValue', 'long_name', 'to_reg_catalogid1',
                      'ref_catalogid1', 'to_reg_catalogid2', 'ref_catalogid2'],
            errors='ignore',
            inplace=True)

# add boolean column for is reference
meta_df['is_reference'] = meta_df['to_reg_acqdate1'].isnull()

# copy/duplicate ref_/to_reg data for reference DEM
for col in meta_df.columns:
    if 'ref' in col:
        new_col = col.replace('ref_', 'to_reg_')
        meta_df.loc[meta_df['is_reference'], new_col] = meta_df.loc[meta_df['is_reference'], col]

meta_df = meta_df.set_index(pd.to_datetime(meta_df['to_reg_acqdate1'].rename('time')))

# add additional metadata
meta_df.attrs = {
    'processed by': 'tlohde',
    'processed on': pd.Timestamp.now().strftime('%Y/%m/%d %H:%M')
}

In [ ]:
demstack = xr.concat(dems, dim='time', combine_attrs='drop').sortby('time')

In [ ]:
demstack = xr.merge([demstack,
                     meta_df.to_xarray()])

In [ ]:
demstack.attrs = {
    'processed by': 'tlohde',
    'processed on': pd.Timestamp.now().strftime('%Y/%m/%d %H:%M'),
    'centreline': gpd.read_file('../data/id1_6685x_-3188046y/line_6685x_-3188046y.geojson').loc[0,'geometry'].wkt
}

TODO
jason briner - long term perspectives of greenland ice sheet position
- if current approx margin configuration is where it _generally_ is during interglacials then it would *make sense* that there are lots of overdeepenings just up-ice.
- Lindbäck, K., R. Pettersson, A. L. Hubbard, S. H. Doyle, D. van As, A. B. Mikkelsen, and A. A. Fitzpatrick. 2015. Subglacial water drainage, storage and piracy beneath the Greenland ice sheet. Geophysical Research Letters 42:7606–14.
see lindback references in ross et al., 2018
- Lindbäck, K., R. Petterson, S. H. Doyle, C. Helanow, P. Jansson, S. S. Kristensen, L. Stenseng, R. Forsberg, and A. L. Hubbard. 2014. High-resolution ice thickness and bed topography of a land-terminating section of the Greenland ice sheet. Earth System Science Data 6:331–38.
is there evidence of *preferential* erosion signature

lake massu
simon mudd, mikael attal - what are lakes? 
rivers **cannot** erode below base level.

TODO
send PN velocity figures and summary paragraph

In [ ]:
ds = xr.open_dataset('../data/id1_6685x_-3188046y/stacked_coregd.zarr/', engine='zarr')

In [ ]:
pf = ds['z'].polyfit(dim='time', deg=1)

In [ ]:
from dask.distributed import Client, LocalCluster
import pandas as pd
# import itslive
import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely import box
import matplotlib.pyplot as plt
# from matplotlib.collections import LineCollection
# from matplotlib.dates import date2num, DateFormatter, YearLocator
# import seaborn as sns
# import xrspatial as xrs
import numpy as np
import xarray as xr
from tqdm import tqdm
from IPython import display
import importlib
# import imagery
import utils
import velocity_helpers

In [ ]:
# _ = importlib.reload(utils)
# _ = importlib.reload(velocity_helpers)
# _ = importlib.reload(imagery)

In [ ]:
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

In [ ]:
## read in lines, and lazily get velocity cubes, centrelines, and robust trends
lines = gpd.read_file('../data/streams_v2.geojson').to_crs(3413)

In [ ]:
V = {}
failed = []
for row in tqdm(lines.sample(3).itertuples()):
    print(f'working on #{row.Index}')
    try:
        V[row.Index] = velocity_helpers.CentreLiner(
            geo=row.geometry,
            buff_dist=500,
            index=row.Index,
            filter_cube=False,
            get_robust_trend=True,
            get_annual_median=False,
            get_rgb=False,
            )
    except Exception as e:
        failed.append((row.Index, e))
        print(f'#{row.Index} did not work because\n{e}')
        continue

## other

In [ ]:
fig, axs = plt.subplots(figsize=[15,8],
                        nrows=2,
                        ncols=4,
                        sharex=True,
                        sharey=True)

for k_v, ax in zip(V.items(), axs.flat):
    k, v = k_v
    velocity_helpers.Plotters.rolling_median(v,
                                             ax=ax,
                                         **{'var': 'v',
                                            'ddt_range': ('335d','395d'),
                                            'ddt_bars': False,
                                            'col': 'cumul_dist',
                                            'window': '90d',
                                            'vals':[500, 1_000, 3_000, 5_000]
                                            })
    _point = utils.shapely_reprojector(v.point, 3413, 4326)
    
    ax.set_title(f'#{k}: {_point.y:.2f}N {_point.x:.2f}W\n{lines.loc[k,"name"]}')
    ax.minorticks_off()

# axs.flat[-2].remove()
axs.flat[-1].remove()

now = pd.Timestamp.now().strftime('%y%m%d_%H%M')+ '_'
directory = '../../GitHub/diary/journal_figures'
filename = 'iceland_outlets_annual_v.png'
fig.savefig(directory + now + filename, bbox_inches = 'tight')

In [ ]:
def centreline_theil_slopes(df, **kwargs):
    
    _var = kwargs.get('var', 'v')
    _x = kwargs.get('x', 'mid_date')
    _date_range = kwargs.get('date_range', (pd.Timestamp('1900-01-01'),
                                            pd.Timestamp.now()))
    _ddt_range = kwargs.get('ddt_range', ('0d', '30d'))
    _mad = kwargs.get('mad', 3)
    
    _df = df.loc[df[_x].between(*_date_range)]    

In [ ]:
idx = ((ds['date_dt'] > pd.Timedelta('335d')) & ((ds['date_dt'] <= pd.Timedelta('395d')))).compute()

from scipy.stats.mstats import theilslopes
from scipy.stats import linregress

x = (ds.mid_date[idx] - ds.mid_date[idx].min()) / pd.Timedelta('365.25d')
y = ds['v'][idx,50,50].compute()

nan_idx = y.isnull()

x = x[~nan_idx]
y= y[~nan_idx]

theil_result = theilslopes(y=y, x=x)
lr_result = linregress(x=x, y=y)


X = np.arange(0,30)
theil_fit = (X * theil_result.slope) + theil_result.intercept
lr_fit = (X * lr_result.slope) + lr_result.intercept

fig, ax = plt.subplots()
ax.scatter(x,y)

ax.plot(X, theil_fit, c='r', label='theilslopes')
ax.plot(X, lr_fit, c='g', label='lr')
ax.legend()

In [ ]:
fig, axs = plt.subplots(figsize=[15,8], nrows=5, ncols=3)
# get_velocity.Plotters.date_dt_bars(V[13],
#                                    ('1d','15d'),
#                                    ax=ax,
#                                    **{'vals':[5_000,1_000]})

get_velocity.Plotters.rolling_median(V[13],
                                     ('335d','395d'),
                                     ax=ax,
                                     ddt_bars=False,
                                     **{'var': 'v',
                                        'col': 'cumul_dist',
                                        'window': '180d',
                                        'vals':[1_000, 3_000, 5_000, 10_000]})

- look at underlying dodgy velocity images
TODO look at GRIMpP and MEASUREs

TODO
- PN thinks that relative change is sufficient to *control* for variations in glacier geometry (read: width/size).
- most interested in *relative* change of velocity over final few kilometers.
    - i.e. 

- it all comes from the same raw processed v data

## imagery

In [ ]:
study_sites = gpd.read_file('../data/potential_study_sites_v1.geojson')
study_sites = study_sites.loc[study_sites['notes'].isin(['yes', 'maybe'])]
study_sites.reset_index(drop=True, inplace=True)
study_sites_5k = study_sites.buffer(5_000).to_crs(4326)

In [ ]:
items = imagery.get_annual_median_mosaic(
    study_sites.loc[0,'geometry'],
    buffer_dist=15_000,
    src_crs=study_sites.crs,
    target_crs=4326,
    timeperiod='2023-01-01/2024-01-01',
    months=[8,9],
    )

In [ ]:
(xrs.multispectral.true_color(
    *items.squeeze()
    .transpose('band', 'y', 'x'))
 .plot.imshow(rgb='band')
)

In [ ]:
import stackstac
stackstac.stack(items, assets=['B04', 'B03', 'B02'])